In [ ]:
# system/os/regex and basic math functions
import os
import re
import sys
import math
import json
import time
import string
import dateutil
import datetime as dt
from itertools import chain

In [ ]:
# Set logging level
import logging
try:
    kwargs = {'level':getattr(logging, LOG_LEVEL)}
    print(f'set LOG_LEVEL="{LOG_LEVEL}"')
except NameError:
    kwargs = {'level':logging.INFO}
    print('Set LOG_LEVEL="WARNING" before running the import file to reduce the clutter.')
try:
    kwargs['format'] = LOG_FORMAT
    print(f'set LOG_FORMAT="{LOG_FORMAT}"')
except NameError:
    kwargs['format'] = "%(levelname)s::%(message)s"
    print('Set LOG_FORMAT="%(levelname)s::%(message)s" to change log format.')

logging.basicConfig(**kwargs)
logger = logging.getLogger('notebook')
del kwargs

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# IPython display convenience stuff
try:
    from IPython.display import HTML, display, display_html, display_javascript
    from IPython import __version__ as ipythonversion
    import ipywidgets
    print("IPython: {}".format(ipythonversion))
except ImportError:
    pass

In [ ]:
try:
    # numpy for matrix algebra
    import numpy as np
    os.environ['NUMEXPR_MAX_THREADS'] = '20'
    print("Numpy (np): {}".format(np.version.full_version))
except ImportError:
    pass

In [ ]:
try:
    # scipy for probability distributions and some statistical tests
    import scipy as sp
    import scipy.stats as stats
    print("Scipy (sp, stats): {}".format(sp.version.full_version))
except ImportError:
    pass

In [ ]:
try:
    # pandas for data manipulation
    import pandas as pd
    print("Pandas (pd): {}".format(pd.__version__))

    def fmt_float(float_in):
        if np.isnan(float_in):
            return 'NaN'
        if float_in//1 == float_in:
            return '{:.0f}'.format(float_in)
        return '{:.3f}'.format(float_in)
    pd.set_option('max_rows', 100, 'max_columns', 100)
    pd.set_option('display.notebook_repr_html', True, 'float_format', fmt_float)
    
    def latex_format(num_in):
        """Format numbers for Latex tables"""
        try:
            num_in = float(num_in)
            num_dig = np.log10(abs(num_in)) + 1
            if num_in == 0:
                return "0"
            if num_dig >= 3:
                return f"{int(num_in):,d}"
            elif num_dig >= 1:
                return f"{num_in:2.1f}"
            return f"{num_in:1.3f}"
        except ValueError:
            return str(num_in)
    
    def S(df, cols=None, keep_dups=False):
        """S splits strings, and if called with a df input, interpolates variable names.
        
        Example::
            S('gvkey datadate') # --> ['gvkey', 'datadate']
            df.S('gvk* datad* num*') # --> ['gvkey', 'datadate', 'num_words', 'num_sentences']
        """
        if isinstance(df, str):
            cols = df
        if isinstance(cols, str):
            new_cols = []
            for col in cols.split():
                if '*' in col or '?' in col:
                    matcher = re.compile(r'\b'+col.replace('*', '.*').replace('?', '.')+r'\b', re.I)
                    new_cols.extend([c for c in df.columns if matcher.search(c)])
                else:
                    new_cols.append(col)
            cols = new_cols
        return cols if keep_dups else list(dict(zip(cols, cols)))
    
    # monkeypatch C into DataFrame
    pd.DataFrame.S = S
except (ImportError, ModuleNotFoundError):
    pass

In [ ]:
try:
    # matplotlib for plotting and pyplot for MATLAB-style API
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    mpl.style.use('seaborn-notebook')
    plt.rcParams['figure.figsize'] = (15, 5) 
    print("MatPlotLib (mpl, plt): {}".format(mpl.__version__))
except ImportError:
    pass

In [ ]:
try:
    # Seaborn for pretty plotting
    import seaborn as sns
    print("Seaborn (sns): {}".format(sns.__version__))
except ImportError:
    pass

In [ ]:
try:
    # Scikit Learn for more regressions
    import sklearn as sk
    print("Scikit-Learn (sk): {}".format(sk.__version__))
except ImportError:
    pass

In [ ]:
try:
    # statsmodels for econometrics
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    print("Statsmodels (sm): {}".format(sm.__version__))
except (ImportError, AttributeError):
    pass

In [ ]:
try:
    # patsy for making formulas
    import patsy as pt
    print("Patsy (pt): {}".format(pt.__version__))
except ImportError:
    pass

In [ ]:
try:
    # SQLAlchemy for relational db management
    import sqlalchemy as sa
    print("SQLAlchemy (sa): {}".format(sa.__version__))
except ImportError:
    pass

In [ ]:
try:
    # Gensim for textual analysis
    import gensim
    print("Gensim: {}".format(gensim.__version__))
except ImportError:
    pass

In [ ]:
try:
    # TQDM for progress bar outputs
    from tqdm.notebook import tqdm
except ImportError:
    def tqdm(thing, *args, **kwargs):
        return thing

In [ ]:
try:
    # SAS7BDAT for reading SAS files
    from sas7bdat import SAS7BDAT as SASdb
except ImportError:
    pass

In [ ]:
try:
    from bs4 import BeautifulSoup
except ImportError:
    pass

In [ ]:
try:
    from pyedgar.utilities import edgarweb
except (ImportError, ModuleNotFoundError):
    class _o_(object):
        def edgar_links(*args, **kwargs):
            return ''
    edgarweb = _o_()

In [ ]:
# sas7bdat for reading SAS created databases
SAS_ZERO = dt.datetime(1960,1,1)
MIN_DATE = dt.datetime(1900, 1, 1)
MAX_DATE = dt.datetime(2030, 1, 1)

def sas_date_to_datetime(df_col):
    return pd.to_timedelta(df_col, unit='d') + SAS_ZERO

TD_DAY = pd.Timedelta(days=1)
TD_YEAR = pd.Timedelta(days=1) * 365

In [ ]:
# print("linkhead(df, n=5, title='', fields=None, cik='cik', accession='accession')")
def linkhead(df, n=5, title='', fields=None, cik='cik', accession='accession', return_df=False):
    """
    Displays top rows of a dataframe, and includes
    links to the HTML and FTP websites if CIK and Accession are found.
    """
    if len(df) == 0:
        display_html(df[fields or df.columns].assign(link='').to_html(), raw=True)
        return
        
    w = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', None)
    
    if fields is None:
        fields = list(df.columns)
    
    dfn = df.head(n).copy()        
    
    if cik in dfn.columns:
        linkstr, i = 'links', 0
        while linkstr in dfn.columns:
            linkstr = 'links%d' % i
            i += 1
        dfn[linkstr] = dfn.apply(lambda row: edgarweb.edgar_links(row[cik], row[accession]), axis=1)
        fields.append(linkstr)
    
    html = f"<h4>{title}</h4>" if title else ''
    html += dfn[fields].to_html(escape=False, index=False, na_rep="")
    
    display_html(html, raw=True)
    pd.set_option('display.max_colwidth', w)
    
    if return_df: 
        return dfn

In [ ]:
# print("timehist(dtseries_or_df, time_variable='year', y_tic_number=4, x_tic_skip=0, *args, **kwargs)")
def timehist(dtseries_or_df, time_variable='year',
             y_tic_number=4, x_tic_skip=0,
             width=.9, ax=None, skip_retick=None,
             label=None, 
             *args, **kwargs):
    """
    Historgam of observations per time period.
    First tries: dtseries_or_df.dt.time_variable
    Failing that, does dtseries_or_df.value_counts()
    Sends args and kwargs to figure.
    """
    if ax is not None and skip_retick is None:
            skip_retick = True
    skip_retick = skip_retick or False
    
    x_tic_skip += 1
    
    if not skip_retick:
        sns.set_style('darkgrid')
        sns.set_context('talk', rc={'patch.linewidth': 0, 'patch.edgecolor': 'k', 'patch.facecolor': 'k'})
    
    _d = dtseries_or_df
    try:
        _d = _d.dt.__getattribute__(time_variable)
    except:
        try:
            _d = _d[time_variable]
        except:
            pass
    _g = _d.value_counts().sort_index()
    if len(_g) > 1000:
        logger.error("ERROR: You are trying to plot something with too many levels. Don't do that.")
        return 
    
    if ax is None:
        if 'figsize' not in kwargs:
            kwargs['figsize'] = (13,2)
        plt.figure(*args, **kwargs)
        ax = plt.gca()
        # If ax is none, assume kwargs are for figure generation.
        kwargs = {}
    
    ax.bar(_g.index, _g, width=width, label=label, **kwargs)
    
    if not skip_retick:
        # Format and label X axis
        ax.set_xlim(left=_g.index.min()-0.5, right=_g.index.max()+0.5)
        _t = _g.index[::x_tic_skip]
        ax.set_xticks(_t)
        ax.set_xticklabels(map(str, _t), rotation='vertical')

        # Label Y Axis
        tene = math.log10(_g.max())//1-1
        topnum = math.ceil(_g.max() / 10**tene)
        ax.set_yticks([(topnum * i // y_tic_number)*10**tene for i in range(y_tic_number, 0, -1)])
    
    return ax

In [ ]:
# print("timeqtrhist(dtseries_or_df, dt_variable='datadate', y_tic_number=4, x_tic_skip=0, *args, **kwargs)")
def timeqtrhist(dtseries_or_df, dt_variable='datadate',
                y_tic_number=4, x_tic_skip=0,
                width=.23, ax=None, skip_retick=False,
                label=None,
                *args, **kwargs):
    """
    Historgam of observations per quarter.
    First tries: dtseries_or_df[dt_variable]
    Failing that, does dtseries_or_df.value_counts()
    Sends args and kwargs to figure.
    """
    x_tic_skip += 1
    sns.set_style('darkgrid')
    sns.set_context('talk', rc={'patch.linewidth': 0, 'patch.edgecolor': 'k', 'patch.facecolor': 'k'})
    _d = dtseries_or_df
    try:
        _d = _d[dt_variable].copy()
    except:
        pass
    try:
        _q = _d.dt.to_period('Q')
        _g = _q.value_counts().sort_index().reset_index()
        _g.columns = 'date nobs'.split()
        _g['xloc'] = _g.date.dt.year + _g.date.dt.quarter/4
        _g4 = _g.copy()
        _g4.loc[_g4.date.dt.quarter!=4, 'nobs'] = 0
    except:
        logger.error("ERROR: Need to pass in a date-time series.")
        return 
    
    if ax is None:
        if 'figsize' not in kwargs:
            kwargs['figsize'] = (13,2)
        plt.figure(*args, **kwargs)
        ax = plt.gca()
    
    ax.bar(_g.xloc,
           _g.nobs,
           width=width, label=None)
    ax.bar(_g4.xloc,
           _g4.nobs,
           width=width, label=label)
    
    if not skip_retick:
        # Format and label X axis
        ax.set_xlim(left=_g.xloc.min()-0.25, right=_g.xloc.max()+0.25)
        _t = _g[_g.date.dt.quarter==4].date.dt.year[::x_tic_skip]
        # Make a plot with major ticks that are multiples of 20 and minor ticks that
        # are multiples of 5.  Label major ticks with '%d' formatting but don't label
        # minor ticks.
        ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))
        ax.xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.0f'))
        plt.setp(ax.get_xticklabels(), rotation=30)

        # For the minor ticks, use no labels; default NullFormatter.
        ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(.25))
        
        #ax.set_xticks(_t)
        #ax.set_xticklabels(map(str, _t), rotation='vertical')

        # Label Y Axis
        tene = math.log10(_g.nobs.max())//1-1
        topnum = math.ceil(_g.nobs.max() / 10**tene)
        ax.set_yticks([(topnum * i // y_tic_number)*10**tene for i in range(y_tic_number, 0, -1)])
    
    return ax

In [ ]:
def savefig(file_name, fig, *args, make_name_unique=False, root_dir=None, **kwargs):
    """
    Save figure to GLOB.FIGURE_DIR.
    
    If no extension is added, PNG is assumed (at default 300 DPI).
    
    If make_name_unique flag is True, Y-M-D_H-M-S is added to filename.
    
    Returns file path that was created.
    """
    _fname, _ext = os.path.splitext(file_name)
    if not _ext:
        _ext = '.png'
    
    file_path = os.path.join(root_dir or config.FIGURE_DIR, _fname + _ext)
    
    if make_name_unique:
        file_path = os.path.join(GLOB.FIGURE_DIR,
                                 f"{_fname}_{dt.date.today():%Y-%m-%d_%H-%M-%S}{_ext}")

    default_kwargs = {
        'bbox_inches': 'tight',
        'pad_inches': 0.1,
        'transparent': True
    }
    
    kwargs = {**default_kwargs, **kwargs}
    
    if _ext == '.png':
        if 'dpi' not in kwargs:
            kwargs['dpi'] = 300
    
    fig.savefig(file_path, *args, **kwargs)
    
    return file_path

In [ ]:
def filehead(path, *args, n=5, **kwargs):
    with open(path, 'r', *args, **kwargs) as fh:
        for i in range(n):
            print(fh.readline(), end='')

In [ ]:
# 3rd party package imports
%matplotlib inline

In [ ]:
# project imports
_code_dir = os.path.abspath('.')
for i in range(100):
    if '{{ cookiecutter.python_libname }}' in os.listdir(_code_dir):
        break
    _code_dir = os.path.dirname(_code_dir)

sys.path.append(_code_dir)
import {{ cookiecutter.python_libname }}
from {{ cookiecutter.python_libname }} import config
